In [1]:
from sortedcontainers import SortedSet
from math import sqrt
## https://stackoverflow.com/questions/8875706/heapq-with-custom-compare-predicate/8875823
# Priority Queue with custom comparator
class MyPriorityQueue(object):
    # compare1 is the function that will return f(n)-> sort in ascending order of f(n)
    # compare2 is the function that will return g(n)-> sort in ascending order of g(n) if f(n) is same
    def __init__(self, current_heuristic, target):
        self.current_heuristic = current_heuristic
#         self._data = []
        self._data = SortedSet()
        self.target = target
        self.g = dict()
        self.h = dict()
    def __len__(self):
        return len(self._data)
    #### It is required to update g(n) and h(n) before pushing the node into the priority queue
    def push(self, item):
        node = (self.calculate_priority(item), self.get_heuristic(item), item)
        self._data.add(node)
#         heapq.heappush(self._data, (self.calculate_priority(item), self.get_heuristic(item), item))
    def pop(self):
        node = self._data.pop(0)[2]
        return node
#         return heapq.heappop(self._data)[2]
    def manhattan_distance(self, x, y): # Manhattan Distance -> heuristic = |x1-x2| + |y1-y2|
        return abs(x[0]-y[0])+abs(x[1]-y[1])
    def euclidean_distance(self, x, y): # Euclidean Distance -> heuristic = sqrt( (x1-x2)**2 + (y1-y2)**2 )
        return sqrt( (x[0]-x[1])**2 + (y[0]-y[1])**2 )
    def chebyshev_distance(self, x, y): # Chebyshev Distance -> heuristic = max(|x1-x2|, |y1-y2|)
        return max(abs(x[0]-x[1]), abs(y[0]-y[1]))
    def get_heuristic(self, x):  # returns the value of heuristic based on the heuristic function
        measure = self.current_heuristic
        if measure == 'm': # if h is manhattan
            self.h[x] = self.manhattan_distance(x, self.target)
        elif measure == 'e': # if h is euclidean
            self.h[x] = self.euclidean_distance(x, self.target)
        elif measure == 'c': # if c is chebyshev
            self.h[x] = self.chebyshev_distance(x, self.target)
        return self.h[x]
    def calculate_priority(self, x): # calculates priority for each cell - distance from cell to target
#         print("*******x: ",x)
#         print("self.g: ",self.g)
#         print("g: ",self.g[x])
        return self.g[x] + self.get_heuristic(x)

class GridWorld:
    def __init__(self, grid, start, target, n, m, agent_grid = None ):
        self.n = n # denotes the number of rows
        self.m = m # denotes the number of columns
        self.start = start # denotes the start coordinate of the grid
        self.target = target # denotes the target coordinate of the grid
        self.trajectory = [] # denotes the trajectory of the agent in the grid
        self.grid = grid # denotes the full knowledge of the grid
        if agent_grid is None:
            self.agent_grid = np.full((n,n),'.')
        else:
            self.agent_grid = agent_grid
#         self.agent_grid =  (agent_grid or np.full((n,n),'.')) # denotes the current knowledge of the grid
        self.avg_num_cells_processed = 0
#         self.g = dict() # denotes the g value -> distance from source to current node
#         self.h = dict() # denotes the h value -> estimation of distance from current node to the target
#     def manhattan_distance(self, x, y): # Manhattan Distance -> heuristic = |x1-x2| + |y1-y2|
#         return abs(x[0]-y[0])+abs(x[1]-y[1])
#     def euclidean_distance(self, x, y): # Euclidean Distance -> heuristic = sqrt( (x1-x2)**2 + (y1-y2)**2 )
#         return sqrt( (x[0]-x[1])**2 + (y[0]-y[1])**2 )
#     def chebyshev_distance(self, x, y): # Chebyshev Distance -> heuristic = max(|x1-x2|, |y1-y2|)
#         return max(abs([0]-x[1]), abs(y[0]-y[1]))
#     def get_heuristic(self, x, measure):  # returns the value of heuristic based on the heuristic function
#         if measure == 'm': # if h is manhattan
#             self.h[x] = sel.manhattan(self, x, y)
#         elif measure == 'e': # if h is euclidean
#             self.h[x] = self.eucliden_distance(self, x, y)
#         elif measure == 'c': # if c is chebyshev
#             self.h[x] = self.chebyshev_distance(self, x, y)
#         return self.h[x]
#     def calculate_priority(self, x, h, g): # calculates priority for each cell - distance from cell to target
# #         print("g: ",self.g)
# #         print("x: ",x)
# #         print("self.g[x]: ",self.g[x])
#         return g + self.get_heuristic(self, x, self.target, h)
    def a_star(self, source, current_heuristic = 'm'):
        dirx = [-1, 1, 0, 0] ## calculates the neighbours of the current node
        diry = [0, 0, 1, -1] ## calculates the neighbours of the current node
#         print("source: ",source)
        visited = set() ## contains the nodes that have already been added in priority queue
        closed_list = set() ## contains the nodes that have been popped from priority queue
        #self.g = dict()
        open_list = MyPriorityQueue(current_heuristic, self.target) ## stores the visited nodes and pops it out according to it's priority
        planned_path = [] ## Stores the path that the agent has planned according to it's current knowledge
        open_list.g[source]=0
        open_list.calculate_priority( source)
        open_list.push( source) ## This function runs in O(logn)
        visited.add(source)#visited[source] = 1
        parent = {} ## stores the parent of each node that is visited
        parent[source] = None
        while(len(open_list)>0):
            curr = open_list.pop()
            if(curr in closed_list):
                continue
            self.avg_num_cells_processed = self.avg_num_cells_processed + 1
            closed_list.add(curr)
            if(curr[0] == self.target[0] and curr[1] == self.target[1]):
                break
            for i in np.arange(4):
                childx = curr[0] + dirx[i]
                childy = curr[1] + diry[i]
                child = (childx, childy)
                if(childx>=0 and childx<self.m and childy>=0 and childy<self.n and (child not in closed_list) and self.agent_grid[childx][childy]!='X'):
                    if(child not in visited):
                        visited.add(child)
                        parent[child] = curr
                        open_list.g[child] = open_list.g[curr]+1
                        open_list.calculate_priority(child)
                        open_list.push(child) #heapq.heappush(open_list, child)
                    else:
                        if open_list.g[curr]+1<open_list.g[child]:
                            parent[child] = curr
                            open_list._data.discard(child)
                            open_list.g[child] = open_list.g[curr]+1
                            open_list.calculate_priority(child)
                            open_list.push(child)
#                             temp = MyPriorityQueue(current_heuristic, self.target)
#                             tempcurr = open_list.pop()
#                             while( len(open_list)>0 and tempcurr != child):
#                                 temp.g[tempcurr] = open_list.g[tempcurr]
#                                 temp.calculate_priority(tempcurr)
#                                 temp.push(tempcurr)
#                                 tempcurr = open_list.pop()
#                             open_list.g[child] = open_list.g[curr]+1
#                             open_list.calculate_priority(child)
#                             open_list.push(child)
#                             while( len(temp)>0 ):
#                                 tempcurr = temp.pop()
#                                 open_list.push(tempcurr)
        if(self.target not in visited):
            return []
        curr = self.target
#         print("$$$$$$$source: ",source)
#         print("agent_grid: \n",self.agent_grid)
#         print("parent: ",parent)
        while(curr != source):
#             print("curr: ",curr,"parent: ",parent[curr])
            planned_path.append(curr)
            curr = parent[curr]
        planned_path.append(source)
        return planned_path[::-1]
    
    
    def check_planned_path(self, planned_path):
        dirx = [-1, 1, 0, 0]
        diry = [0, 0, -1, 1]
        n = len(planned_path)
        traversed_path = []
        for i in np.arange(n):
            currx = planned_path[i][0]
            curry = planned_path[i][1]
            if(grid[currx][curry] == 'X'):
                break
            traversed_path.append((currx,curry))
            for i in np.arange(4):
                childx = currx + dirx[i]
                childy = curry + diry[i]
                if(childx>=0 and childx<self.m and childy>=0 and childy<self.n):
                    self.agent_grid[childx][childy] = self.grid[childx][childy]
        return traversed_path
    
    ## This is the function that computes the path that repeated a-star traverses
    def compute_path(self):
        path = []
        curr = self.start
        while(curr != self.target):
            planned_path = self.a_star(curr)
            if( len(planned_path) == 0 ):
                return []
            traversed_path = self.check_planned_path(planned_path)
            n = len(traversed_path)
            path.append(traversed_path)
            if(traversed_path[n-1] == self.target):
                break
            curr = traversed_path[n-1]
        # self.combine_path(path)
        return path
                        

In [27]:
import random
import numpy as np

def generateRandomGrid(p):
    matrix_string = "."
    for i in range(1,10200):
        if(random.uniform(0, 1)<=p):
            matrix_string+='X'
        else:
            matrix_string+='.'
    matrix_string+='.'
    return matrix_string
#     print(matrix_string)
def generateData(p):
    grids = []
    count = 0
    for i in range(0,2500):
        if count>=1000:
            break
        grid1 = generateRandomGrid(p)
        i = 0
        grid2 = [] ## matrix that contains full knowledge of gridworld
        while(i<101*101):
            grid2.append(list(grid1[i:i+101]))
            i = i + 101
        grid2 = np.array(grid2)
        gw = GridWorld(grid2,(0,0),(100,100),101,101,agent_grid=grid2)
        path=gw.a_star((0,0),'m')
        if len(path)>0:
            grids.append(grid1)
#             print(gw.grid)
            count+=1
#     print(grids)
    with open('p'+str(p)+'.txt', 'w') as f:
        print(str(p)+": ",len(grids))
        for grid in grids:
            f.write(grid)
            f.write('\n')
p_values = np.linspace(0,0.3,31)
for i in range(1,31):
#     print(p_values[i])
    generateData(round(p_values[i],2))

0.01:  1000
0.02:  1000
0.03:  1000
0.04:  1000
0.05:  1000
0.06:  1000
0.07:  1000
0.08:  1000
0.09:  1000
0.1:  1000
0.11:  1000
0.12:  1000
0.13:  1000
0.14:  1000
0.15:  1000
0.16:  1000
0.17:  1000
0.18:  1000
0.19:  1000
0.2:  1000
0.21:  1000
0.22:  1000
0.23:  1000
0.24:  1000
0.25:  1000
0.26:  1000
0.27:  1000
0.28:  1000
0.29:  1000
0.3:  1000


In [13]:
grids

NameError: name 'grids' is not defined

In [62]:
# p_values = np.linspace(0,1,51)
p_values = [0.02]
for i in p_values:
    with open('C:/Users/Naishal/Desktop/Grid_data/'+'p'+str(i)+'.txt') as f:
        gridWorld = f.readlines()
        #A*
print(gridWorld[0])
with open('trial.txt', 'w') as f:
    f.write(gridWorld[1])

..............X.......................................X...............X.................................X............................X..............................................................................................................X.........................X..............................................................................X...........................................................X..............................................................................................................................X..................................................................................X.........X.....................X....................................................................X.................................X...........................X..........X..................X..................................................................X...............................................................................................................X........

In [70]:
p_values = np.linspace(0,0.3,31)
print(len(p_values))
for i in range(1,31):
    print(p_values[i])

31
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3


In [7]:
import random
import numpy as np

def generateRandomGrid(p):
    matrix_string = "."
    for i in range(1,399):
        if(random.uniform(0, 1)<=p):
            matrix_string+='X'
        else:
            matrix_string+='.'
    matrix_string+='.'
    print(len(matrix_string))
    return matrix_string
    
def generateData(p):
    grids = []
    count = 0
    for i in range(0,100):
        if count>=30:
            break
        grid1 = generateRandomGrid(p)
        i = 0
        grid2 = [] ## matrix that contains full knowledge of gridworld
        while(i<20*20):
            grid2.append(list(grid1[i:i+20]))
            i = i + 20
        grid2 = np.array(grid2)
        gw = GridWorld(grid2,(0,0),(19,19),20,20,agent_grid=grid2)
        path=gw.a_star((0,0),'m')
        if len(path)>0:
            grids.append(grid1)
#             print(gw.grid)
            count+=1
#     print(grids)
    with open('p'+str(p)+'.txt', 'w') as f:
        print(str(p)+": ",len(grids))
        for grid in grids:
            f.write(grid)
            f.write('\n')
# p_values = np.linspace(0,0.3,31)
# for i in range(1,31):
# #     print(p_values[i])
generateData(round(0.3,2))

400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
0.3:  30


In [2]:
#hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
import random
import pickle
import numpy as np
dim = 100
srcs = []
tgts = []

def generateRandomGrid(p=0.3):
    matrix_string = ""
    for i in range(0,dim*dim):
        if(random.uniform(0, 1) <= p):
            matrix_string+='X'
        else:
            terrain = random.randint(1,3)
            if(terrain == 1):
                matrix_string += 'P'
            elif(terrain == 2):
                matrix_string += 'H'
            else:
                matrix_string += 'F'                
    return matrix_string
#     print(matrix_string)
def generateData(p=0.3):
    grids = []
    
    count = 0
    for i in range(0,2500):
        if count>=100:#number of grids to be generated
            break
        grid1 = generateRandomGrid(p)
        i = 0
        grid2 = [] ## matrix that contains full knowledge of gridworld
        while(i<dim*dim):
            grid2.append(list(grid1[i:i+dim]))
            i = i + dim
        grid2 = np.array(grid2)
        
        while True:
            src = (random.randint(0, dim-1),random.randint(0, dim-1))
            sindex = dim*src[0] + src[1]
            if grid1[sindex] != 'X':
                break
            
        while True:
            tgt = (random.randint(0, dim-1),random.randint(0, dim-1))
            tindex = dim*tgt[0] + tgt[1]
            if grid1[tindex] != 'X':
                if src == tgt:
                    continue
                break
            
        gw = GridWorld(grid2,src,tgt,dim,dim,agent_grid=grid2)
        path=gw.a_star(src,'m')
        if len(path)>0:
            grids.append(grid1)
            srcs.append(src)
            tgts.append(tgt)
#             print(gw.grid)
            count+=1
#     print(grids)
    with open('p'+str(p)+'_100.txt', 'w') as f:
        print(str(p)+": ",len(grids))
        for grid in grids:
            f.write(grid)
            f.write('\n')
generateData()
with open('project-3-srcs_100.pkl', 'wb') as f:
        pickle.dump(srcs, f)
        
with open('project-3-tgts_100.pkl', 'wb') as f:
        pickle.dump(tgts, f)

0.3:  100


In [53]:
random.randint(0,2)

2

In [ ]:
#hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
import random
import pickle
import numpy as np
dim = 2
srcs = []
tgts = []

def generateRandomGrid(p=0.3):
    matrix_string = ""
    for i in range(0,dim*dim):
        if(random.uniform(0, 1) <= p):
            matrix_string+='X'
        else:
            terrain = random.randint(1,3)
            if(terrain == 1):
                matrix_string += 'P'
            elif(terrain == 2):
                matrix_string += 'H'
            else:
                matrix_string += 'F'                
    return matrix_string
#     print(matrix_string)
def generateData(p=0.3):
    grids = []
    
    count = 0
    for i in range(0,2500):
        if count>=1:#number of grids to be generated
            break
        grid1 = generateRandomGrid(p)
        i = 0
        grid2 = [] ## matrix that contains full knowledge of gridworld
        while(i<dim*dim):
            grid2.append(list(grid1[i:i+dim]))
            i = i + dim
        grid2 = np.array(grid2)
        
        while True:
            src = (random.randint(0, dim-1),random.randint(0, dim-1))
            sindex = dim*src[0] + src[1]
            if grid1[sindex] != 'X':
                break
            
        while True:
            tgt = (random.randint(0, dim-1),random.randint(0, dim-1))
            tindex = dim*tgt[0] + tgt[1]
            if grid1[tindex] != 'X':
                if src == tgt:
                    continue
                break
            
        gw = GridWorld(grid2,src,tgt,dim,dim,agent_grid=grid2)
        path=gw.a_star(src,'m')
        if len(path)>0:
            grids.append(grid1)
            srcs.append(src)
            tgts.append(tgt)
#             print(gw.grid)
            count+=1
    print(grids)
    print(srcs)
    print(tgts)
#     with open('p'+str(p)+'.txt', 'w') as f:
#         print(str(p)+": ",len(grids))
#         for grid in grids:
#             f.write(grid)
#             f.write('\n')
generateData()
# with open('tstproject-3-srcs.pkl', 'wb') as f:
#         pickle.dump(srcs, f)
        
# with open('project-3-tgts.pkl', 'wb') as f:
#         pickle.dump(tgts, f)